# Homework "6"

Austin Gill

In [ ]:
# %config InlineBackend.figure_format = 'retina'
%config InlineBackend.figure_format = 'svg'
%matplotlib inline

import sympy
sympy.init_printing()

import numpy as np
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse
# Apparently, SNS stands for "Samuel Norman Seaborn", a fictional
# character from The West Wing
import seaborn as sns

sns.set()

## Problem 18.1

Let the domain be the rectangle $0 \leq x \leq 15$ and $0 \leq y \leq 10$. Place the start position at $(0,5)$. Place the end position at $(15,5)$. Assume you have circular obstacles centered at $(6,4)$ with radius $2$ and at $(8,6)$ with radius $3$. Find a potential function which can navigate the robot from the start to the end position.

1. Plot the resulting path in Python with obstacles included in the map.
2. Compare to the Wavefront approach.

We have $q_{goal}$ at $(15, 5)$, so we build

$$U_a(q) = \begin{cases}
    \frac{1}{2}\gamma \mathrm{d}^2(q, q_{goal}), &\mathrm d(q, q_{goal}) \leq \mathrm d_{goal}^* \\
    \mathrm d_{goal}^* \gamma \mathrm d(q, q_{goal}) - \frac{1}{2} \gamma \left(\mathrm d_{goal}^*\right)^2, &\mathrm d(q, q_{goal}) > \mathrm d_{goal}^* \\
\end{cases}$$

where $\mathrm d_{goal}^*$ is the distance from the goal at which we switch from a linear potential function to a quadratic, and $\gamma$ is another configurable parameter with $\mathrm d$ being the usual euclidean distance function.

Then for each of the two obstacles, we have a

$$U_i = \begin{cases}
    \frac{1}{2}\eta \left(\frac{1}{\mathrm D_i(q)} - \frac{1}{Q^*}\right), &\mathrm D_i(q) \leq Q^* \\
    0, &\mathrm D_i(q) > Q^* \\
\end{cases}$$

where $\mathrm D_i(q)$ is the distance to the $i$th obstacle and $Q^*$ is the cutoff distance after which the obstacle should not impact the potential field.

For this particular problem, we have

$$U_a(x, y) = \begin{cases}
    \gamma \frac{1}{2} \left((x - 15)^2 + (y - 5)^2\right), &\mathrm d(q, q_{goal}) \leq \mathrm d_{goal}^* \\
    \mathrm d_{goal}^* \gamma \sqrt{(x - 15)^2 + (y - 5)^2} - \gamma \frac{1}{2} \left(\mathrm d_{goal}^*\right)^2, &\text{ else}
\end{cases}$$

$$U_1 = \begin{cases}
    \eta\frac{1}{2}\left(\frac{1}{(x - 6)^2 + (y - 4)^2 - 4} - \frac{1}{Q^*}\right), &\mathrm D_1(q) \leq Q^* \\
    0, &\text{ else}
\end{cases}$$

$$U_2 = \begin{cases}
    \eta\frac{1}{2}\left(\frac{1}{(x - 8)^2 + (y - 6)^2 - 9} - \frac{1}{Q^*}\right), &\mathrm D_2(q) \leq Q^* \\
    0, &\text{ else}
\end{cases}$$

and make wild ass guesses for $\gamma$, $\mathrm d_{goal}^*$, $Q^*$, and $\eta$.

First we plot the start point, goal, and obstacles.

In [ ]:
plt.plot(0, 5, '.', label='start')
plt.plot(15, 5, '.', label='goal')
ax = plt.gca()

# Note that the height and width are *not* radii.
ax.add_patch(Ellipse(xy=(8, 6), width=6, height=6, angle=0, color='r'))
ax.add_patch(Ellipse(xy=(6, 4), width=4, height=4, angle=0, color='r'))

plt.xlim(0-0.2, 16)
plt.ylim(0-0.2, 10+0.2)
plt.xlabel('$x$')
plt.ylabel('$y$')
plt.title('Map')
plt.legend()
plt.show()

Then we plot the potential function that we generate. Note that I originally intended to use the piecewise functions from above, but ran into significant problems that I still don't understand. I also wish that the gradient near the obstacle was more gradual than it is.

In [ ]:
x = np.arange(-1, 16, 0.1)
y = np.arange(-1, 11, 0.1)

X, Y = np.meshgrid(x, y)

def U_a(x, y, gamma=1.0):
    """The attractive potential function."""
    d2 = (x - 15)**2 + (y - 5)**2
    return 0.5 * gamma * d2

def U_1(x, y, eta=2.0):
    """The repulsive potential function for the first obstacle."""
    d2 = (x - 6)**2 + (y - 4)**2 - 4
    # Avoid division by zero, but only if not doing symbolic math.
    if not isinstance(x, sympy.Symbol):
        d2[d2 < 0.05] = 0.05
    return 0.5 * eta / d2

def U_2(x, y, eta=2.0):
    """The repulsive potential function for the second obstacle."""
    d2 = (x - 8)**2 + (y - 6)**2 - 9
    # Avoid division by zero, but only if not doing symbolic math.
    if not isinstance(x, sympy.Symbol):
        d2[d2 < 0.05] = 0.05
    return 0.5 * eta / d2

Z = U_a(X, Y) + U_2(X, Y) + U_1(X, Y)

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
# ax.view_init(azim=-90, elev=89)

ax.plot_surface(X, Y, Z, linewidth=0.05)

# ax.set_zlim(0, 10)
ax.set_xlabel('$x$')
ax.set_ylabel('$y$')
ax.set_zlabel('$z$')

plt.show()

I also plot the contours of the potential function. Due to the way I capped the repulsive potentials to avoid division by zero for a nice pretty plot, the top of each obstacle has the same contours as the original attractive potential function. So if we ever stepped inside the obstacle during gradient descent, we'd plot a path through the obstacle towards the goal.

However, I've only capped the repulsive potentials for plotting purposes. Due to the symbolic nature of the gradient, the gradient descent algorithm will still see the infinite peaks along the edge of the obstacle.

In [ ]:
plt.contour(X, Y, Z, levels=20)

plt.plot(15, 5, '.', label='goal')

ax = plt.gca()

# Note that the height and width are *not* radii.
ax.add_patch(Ellipse(xy=(8, 6), width=6, height=6, angle=0, color='r'))
ax.add_patch(Ellipse(xy=(6, 4), width=4, height=4, angle=0, color='r'))

plt.xlim(-1, 16)
plt.ylim(-1, 11)
plt.xlabel('$x$')
plt.ylabel('$y$')
plt.title('Potential function contours')
plt.legend()
plt.show()

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
# ax.view_init(azim=-90, elev=89)

ax.plot_surface(X, Y, Z, linewidth=0, antialiased=False, cmap=sns.cubehelix_palette(dark=0.1, light=0.75, as_cmap=True))

ax.contour(X, Y, Z, 20, zdir='z', offset=-50)
ax.contour(X, Y, Z, zdir='x', offset=-5)
ax.contour(X, Y, Z, 2, zdir='y', offset=15)

ax.set_zlim(-50, 140)
ax.set_xlim(-5, 16)
ax.set_ylim(-1, 15)

ax.set_xlabel('$x$')
ax.set_ylabel('$y$')
ax.set_zlabel('$z$')
plt.title('Potential surface with contours')
plt.show()

Now I need to perform gradient descent to find the path from $(0, 5)$ to $(15, 5)$ that avoids the obstacles. Unfortunately, I cannot find a way to get [`scipy.optimize.minimize`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize.html) to return the path from the initial guess to the minima, so I have to compute the gradient and perform gradient descent myself.

I guess that makes sense, because `scipy.optimize.minimize` is designed to *find* the minima, but in this case we know *exactly* where the minima is be design! What we're interested in is the *path* from the initial point to $q_{goal}$.

So use `sympy` to symbolically create the potential function and compute its gradient.

In [ ]:
x, y = sympy.symbols('x y')

Potential = sympy.Matrix([U_a(x, y) + U_1(x, y) + U_2(x, y)])
Potential

In [ ]:
Gradient = Potential.jacobian([x, y])
Gradient

Then convert the `sympy` functions to `numpy` functions and test some points of interest.

In [ ]:
gradient = sympy.lambdify([x, y], Gradient, 'numpy')
potential = sympy.lambdify([x, y], Potential, 'numpy')

# Should point to (15, 5), which it almost does...
print(gradient(15, 0))
print(gradient(15, 10))

Now that I have nice continuous forms of the potential and gradient, plot the vector field and weep. Unfortunately, it evenly spaces the vectors throughout the domain, which includes the obstacles. I do not feel adventurous enough to try to mask those values out. So this is what you get.

In [ ]:
nabla = -gradient(X, Y)
U, V = nabla[0]

plt.title('Garbage Vector Field')
plt.streamplot(X, Y, U, V, color=np.clip(potential(X, Y).reshape(X.shape), -100, 200), cmap='magma')
plt.show()

I'm a little concerned that the symbolically computed gradients do not point straight towards the minima when evaluated at very simply points, but I'll slam on my "I believe" button for right now. Now perform the simplest gradient descent algorithm you've ever seen to record the path from $(0, 5)$ to $(15, 5)$.

In [ ]:
def descent(gradient, x0, alpha, N):
    """Perform gradient descent on the given gradient."""
    path = [x0]
    for i in range(N):
        x0 = x0 - alpha * gradient(x0[0, 0], x0[0, 1])
        path.append(x0)
        
    return np.array(path).reshape(N+1, 2)

In [ ]:
def plot_path(path, alpha):
    """Plot the given path on the map."""
    plt.plot(*zip(*path), label='path')

    plt.plot(0, 5, '.', label='start')
    plt.plot(15, 5, '.', label='goal')
    ax = plt.gca()

    # Note that the height and width are *not* radii.
    ax.add_patch(Ellipse(xy=(8, 6), width=6, height=6, angle=0, color='r'))
    ax.add_patch(Ellipse(xy=(6, 4), width=4, height=4, angle=0, color='r'))

    plt.xlim(-1, 16)
    plt.ylim(-1, 11)
    plt.xlabel('$x$')
    plt.ylabel('$y$')
    plt.title(r'Gradient Descent with $\alpha={}$'.format(alpha))
    plt.legend()
    plt.show()

In [ ]:
alpha = 0.01
N = 750
x0 = np.array([[0, 5]])

plot_path(descent(gradient, x0, alpha, N), alpha)

Which is *soo* much better than I ever expected! You know me though, I can't not screw with things, so let's do some experiments.

In [ ]:
configs = [(0.02, 400), (0.025, 400), (0.03, 5000)]

for alpha, N in configs:
    plot_path(descent(gradient, x0, alpha, N), alpha)

As you can see, the gradient descent algorithm I chose is *quite* sensitive to the step size I choose. This is expected, because if you step up the gradient around the obstacles, it pushes away from the obstacle harder than the goal attacts. So if we step too far, we'll step up the obstacle and then back away.

In particular, note that the third doesn't even converge in fewer than 5000 iterations!!

Now let's increase the repulsion factor $\eta$ to $10$ and see if it helps.

In [ ]:
Potential = sympy.Matrix([U_a(x, y, gamma=1.0) + U_1(x, y, eta=10.0) + U_2(x, y, eta=10.0)])
Potential

In [ ]:
Gradient = Potential.jacobian([x, y])
Gradient

In [ ]:
gradient2 = sympy.lambdify([x, y], Gradient, 'numpy')

In [ ]:
configs = [(0.04, 400), (0.05, 150), (0.06, 100), (0.1, 100)]

for alpha, N in configs:
    plot_path(descent(gradient2, x0, alpha, N), alpha)

Not only is the gradient descent more robust, it also pushes the path just slightly further from the obstacle.

In [ ]:
alpha = 0.01
N = 750

plot_path(descent(gradient, x0, alpha, N), alpha)
plot_path(descent(gradient2, x0, alpha, N), alpha)